In [ ]:
# ============================================================
#   PROJECT 2 — NBA Scheduling
#   Clean Fixed Version (Questions 1, 2, and 3)
#   !!!! COPY & PASTE THIS ENTIRE FILE !!!!
# ============================================================

import pandas as pd
import gurobipy as gp
from gurobipy import GRB


# ============================================================
# QUESTION 1 — Read schedule + print statistics
# ============================================================

df = pd.read_csv("games.csv")

teams = list(set(df["Home"]).union(set(df["Visitor"])))

print("\n================= QUESTION 1 =================\n")

for i in teams:
    print("-" * 60)
    print(f"Team: {i}")

    # (a) dates when team i played home
    home_dates_i = df.loc[df["Home"] == i, "Date"].tolist()
    print("\n(a) HOME Dates:")
    for d in home_dates_i:
        print(f"   {d}")

    # (b) times i hosted j
    print("\n(b) Times team i hosted team j:")
    home_vs_counts = (
        df[df["Home"] == i]
        .groupby("Visitor")
        .size()
        .reindex(teams, fill_value=0)
    )
    for j, count in home_vs_counts.items():
        if j != i:
            print(f"   vs {j}: {count}")

    # (c) times i played away at j
    print("\n(c) Times team i played AWAY at team j:")
    away_vs_counts = (
        df[df["Visitor"] == i]
        .groupby("Home")
        .size()
        .reindex(teams, fill_value=0)
    )
    for j, count in away_vs_counts.items():
        if j != i:
            print(f"   at {j}: {count}")

    # (d) dates when team i played away
    away_dates_i = df.loc[df["Visitor"] == i, "Date"].tolist()
    print("\n(d) AWAY Dates:")
    for d in away_dates_i:
        print(f"   {d}")

print("\n================= END QUESTION 1 =================\n")


# ============================================================
# QUESTION 2 — Build baseline IP model (constraints (e)-(h))
# ============================================================

print("\n================= QUESTION 2 =================\n")

df = pd.read_csv("games.csv")
teams = list(set(df["Home"]).union(set(df["Visitor"])))

# H_i: dates when team i plays home
H = {i: list(df[df["Home"] == i]["Date"].unique()) for i in teams}

# A_i: dates when team i plays away
A = {i: list(df[df["Visitor"] == i]["Date"].unique()) for i in teams}

# h_ij: number of times i hosts j
h = {(i, j): 0 for i in teams for j in teams if i != j}
home_pairs = df.groupby(["Home", "Visitor"]).size()
for (i, j), val in home_pairs.items():
    if i != j:
        h[i, j] = int(val)

# a_ij: number of times i visits j
a = {(i, j): 0 for i in teams for j in teams if i != j}
away_pairs = df.groupby(["Visitor", "Home"]).size()
for (i, j), val in away_pairs.items():
    if i != j:
        a[i, j] = int(val)

# CREATE MODEL
model = gp.Model("NBA_schedule_IP")

# Decision vars: x[i,j,d] = 1 if i hosts j on date d
x = {}
for i in teams:
    for j in teams:
        if i == j:
            continue
        for d in H[i]:
            if d in A[j]:
                x[i, j, d] = model.addVar(vtype=GRB.BINARY,
                                          name=f"x_{i}_{j}_{d}")

model.update()

# (e) home team plays exactly one game on each home date
for i in teams:
    for d in H[i]:
        model.addConstr(
            gp.quicksum(x[i, j, d] for j in teams if j != i and (i, j, d) in x) == 1
        )

# (f) away team plays exactly one game on each away date
for j in teams:
    for d in A[j]:
        model.addConstr(
            gp.quicksum(x[i, j, d] for i in teams if i != j and (i, j, d) in x) == 1
        )

# (g) times i hosts j must match original schedule
for i in teams:
    for j in teams:
        if i != j:
            model.addConstr(
                gp.quicksum(x[i, j, d] for d in H[i] if (i, j, d) in x) == h[i, j]
            )

# (h) times i plays away at j must match original
for i in teams:
    for j in teams:
        if i != j:
            model.addConstr(
                gp.quicksum(x[j, i, d] for d in H[j] if (j, i, d) in x) == a[i, j]
            )

print("Solving Question 2 baseline model...")
model.optimize()

if model.status == GRB.OPTIMAL:
    print("Baseline (Question 2) model is FEASIBLE.\n")
else:
    print("Baseline (Question 2) model is INFEASIBLE.\n")


# ============================================================
# QUESTION 3 — Add Time Zone Fatigue Constraint
# ============================================================

print("\n================= QUESTION 3 =================\n")

# Time zone index map
tz_map = {
    'Boston Celtics': 0, 'New York Knicks': 0, 'Brooklyn Nets': 0,
    'Philadelphia 76ers': 0, 'Toronto Raptors': 0, 'Miami Heat': 0,
    'Atlanta Hawks': 0, 'Cleveland Cavaliers': 0, 'Detroit Pistons': 0,
    'Indiana Pacers': 0, 'Washington Wizards': 0, 'Charlotte Hornets': 0,
    'Orlando Magic': 0, 'Chicago Bulls': 1, 'Milwaukee Bucks': 1,
    'Houston Rockets': 1, 'Dallas Mavericks': 1, 'Minnesota Timberwolves': 1,
    'Oklahoma City Thunder': 1, 'Memphis Grizzlies': 1,
    'New Orleans Pelicans': 1, 'San Antonio Spurs': 1,
    'Denver Nuggets': 2, 'Phoenix Suns': 2, 'Utah Jazz': 2,
    'Los Angeles Lakers': 3, 'Golden State Warriors': 3,
    'Los Angeles Clippers': 3, 'Sacramento Kings': 3,
    'Portland Trail Blazers': 3
}

# Collect all delta variables for objective
all_deltas = []

for t in teams:
    # All dates team t plays (home or away), sorted chronologically
    all_dates_t = sorted(H[t] + A[t], key=lambda d: pd.to_datetime(d))

    # Determine the timezone expression for each game date
    loc_exprs = {}
    for d in all_dates_t:
        if d in H[t]:
            loc_exprs[d] = tz_map[t]   # home: fixed tz
        else:
            # away: depends on host j
            possible_hosts = [j for j in teams if j != t and (j, t, d) in x]
            loc_exprs[d] = gp.quicksum(tz_map[j] * x[j, t, d] for j in possible_hosts)

    # Create delta vars for adjacent games
    team_deltas = []
    for k in range(len(all_dates_t) - 1):
        d_curr = all_dates_t[k]
        d_next = all_dates_t[k + 1]

        delta = model.addVar(vtype=GRB.CONTINUOUS, name=f"delta_{t}_{k}")
        team_deltas.append(delta)
        all_deltas.append(delta)

        # Linearize |loc_curr - loc_next|
        model.addConstr(delta >= loc_exprs[d_curr] - loc_exprs[d_next])
        model.addConstr(delta >= loc_exprs[d_next] - loc_exprs[d_curr])

    # Fatigue constraint: no sum of two consecutive deltas >= 4
    for k in range(len(team_deltas) - 1):
        model.addConstr(team_deltas[k] + team_deltas[k + 1] <= 3)

# Objective: minimize total travel
model.setObjective(gp.quicksum(all_deltas), GRB.MINIMIZE)

print("Solving Question 3 model with fatigue constraint...")
model.optimize()

if model.status == GRB.OPTIMAL:
    print("A feasible schedule was found (very unlikely).")
elif model.status == GRB.INFEASIBLE:
    print("\n--- MODEL IS INFEASIBLE (EXPECTED FOR QUESTION 3) ---")
    print("No schedule exists that satisfies fatigue + fixed home/away constraints.")
    model.computeIIS()
    model.write("nba_conflict_report.ilp")
    print("IIS written to nba_conflict_report.ilp")
else:
    print("Solver ended with status:", model.status)



================= QUESTION 1 =================

------------------------------------------------------------
Team: Los Angeles Lakers

(a) HOME Dates:
   Mon, Nov 03, 2025
   Wed, Nov 05, 2025
   Tue, Nov 11, 2025
   Sat, Nov 15, 2025
   Mon, Nov 17, 2025
   Wed, Nov 19, 2025
   Thu, Dec 25, 2025

(b) Times team i hosted team j:
   vs Cleveland Cavaliers: 1
   vs Phoenix Suns: 1
   vs Dallas Mavericks: 0
   vs Golden State Warriors: 1
   vs Denver Nuggets: 0
   vs Brooklyn Nets: 1
   vs Toronto Raptors: 1
   vs Chicago Bulls: 0
   vs Atlanta Hawks: 0
   vs Philadelphia 76ers: 0
   vs New York Knicks: 1
   vs Miami Heat: 0
   vs Boston Celtics: 1
   vs Houston Rockets: 0
   vs Milwaukee Bucks: 0

(c) Times team i played AWAY at team j:
   at Cleveland Cavaliers: 0
   at Phoenix Suns: 0
   at Dallas Mavericks: 1
   at Golden State Warriors: 0
   at Denver Nuggets: 1
   at Brooklyn Nets: 0
   at Toronto Raptors: 0
   at Chicago Bulls: 1
   at Atlanta Hawks: 1
   at Philadelphia 76ers: 1


Solving Question 2 baseline model...
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[rosetta2] - Darwin 25.1.0 25B78)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 736 rows, 1024 columns and 4096 nonzeros
Model fingerprint: 0x53e70b1e
Variable types: 0 continuous, 1024 integer (1024 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 430 rows and 584 columns
Presolve time: 0.02s
Presolved: 306 rows, 440 columns, 1386 nonzeros
Variable types: 0 continuous, 440 integer (440 binary)

Root relaxation: objective 0.000000e+00, 400 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       0.0000000    0.00000  0.

In [ ]:
# ============================================================
# QUESTION 4 — Improve schedule by minimizing time-zone travel
# ============================================================

print("\n================= QUESTION 4 =================\n")

# We reuse: df, teams, H, A, h, a, tz_map defined above.

# Build a NEW model for Part 4 (don't reuse the infeasible one from Q3)
model4 = gp.Model("NBA_schedule_Part4")

# Decision vars: x4[i,j,d] = 1 if i hosts j on date d
x4 = {}
for i in teams:
    for j in teams:
        if i == j:
            continue
        for d in H[i]:
            if d in A[j]:
                x4[i, j, d] = model4.addVar(vtype=GRB.BINARY,
                                            name=f"x4_{i}_{j}_{d}")

model4.update()

# (e) For each team i and each home date d in H_i: exactly one home game
for i in teams:
    for d in H[i]:
        model4.addConstr(
            gp.quicksum(x4[i, j, d]
                        for j in teams
                        if j != i and (i, j, d) in x4) == 1,
            name=f"home_once_{i}_{d}"
        )

# (f) For each team j and each away date d in A_j: exactly one away game
for j in teams:
    for d in A[j]:
        model4.addConstr(
            gp.quicksum(x4[i, j, d]
                        for i in teams
                        if i != j and (i, j, d) in x4) == 1,
            name=f"away_once_{j}_{d}"
        )

# (g) For each pair (i,j), total times i hosts j equals h_ij
for i in teams:
    for j in teams:
        if i == j:
            continue
        model4.addConstr(
            gp.quicksum(x4[i, j, d]
                        for d in H[i]
                        if (i, j, d) in x4) == h[i, j],
            name=f"host_count_{i}_{j}"
        )

# (h) For each pair (i,j), total times i is away at j equals a_ij
#     i away at j ⇔ variable x4[j,i,d]
for i in teams:
    for j in teams:
        if i == j:
            continue
        model4.addConstr(
            gp.quicksum(x4[j, i, d]
                        for d in H[j]
                        if (j, i, d) in x4) == a[i, j],
            name=f"away_count_{i}_{j}"
        )

# --------- Time-zone travel structure (no fatigue constraint) ---------

all_deltas4 = []

for t in teams:
    # All dates team t plays (home or away), sorted
    all_dates_t = sorted(H[t] + A[t], key=lambda d: pd.to_datetime(d))

    # Time-zone "location" expression for each of t's game dates
    loc_exprs = {}
    for d in all_dates_t:
        if d in H[t]:
            # t is at home -> fixed tz
            loc_exprs[d] = tz_map[t]
        else:
            # t is away -> tz of the host j
            possible_hosts = [j for j in teams if j != t and (j, t, d) in x4]
            loc_exprs[d] = gp.quicksum(tz_map[j] * x4[j, t, d]
                                       for j in possible_hosts)

    # Delta variables between consecutive games of team t
    for k in range(len(all_dates_t) - 1):
        d_curr = all_dates_t[k]
        d_next = all_dates_t[k + 1]

        delta = model4.addVar(vtype=GRB.CONTINUOUS,
                              name=f"delta4_{t}_{k}")
        all_deltas4.append(delta)

        # Linearize |loc_curr - loc_next|
        model4.addConstr(delta >= loc_exprs[d_curr] - loc_exprs[d_next],
                         name=f"delta_pos_{t}_{k}")
        model4.addConstr(delta >= loc_exprs[d_next] - loc_exprs[d_curr],
                         name=f"delta_neg_{t}_{k}")

# Objective: minimize total time-zone "travel" across all teams
if all_deltas4:
    model4.setObjective(gp.quicksum(all_deltas4), GRB.MINIMIZE)
else:
    model4.setObjective(0.0, GRB.MINIMIZE)

print("Solving Part 4 model (minimize total time-zone travel)...")
model4.optimize()

# ============================================================
# Extract optimized schedule and compare to original
# ============================================================

def compute_tz_travel(df_schedule, teams, tz_map):
    """
    Compute total time-zone travel across all teams,
    and per-team travel, based on consecutive games.
    """
    total_travel = 0
    travel_by_team = {}

    for t in teams:
        games_t = df_schedule[(df_schedule["Home"] == t) |
                              (df_schedule["Visitor"] == t)].copy()

        if games_t.empty:
            travel_by_team[t] = 0
            continue

        games_t["Date"] = pd.to_datetime(games_t["Date"])
        games_t = games_t.sort_values("Date")

        # Build time-zone sequence for team t
        tz_seq = []
        for _, row in games_t.iterrows():
            if row["Home"] == t:
                # home game: location = home arena
                tz_seq.append(tz_map[row["Home"]])
            else:
                # away game: location = opponent's arena
                tz_seq.append(tz_map[row["Home"]])

        travel_t = 0
        for k in range(len(tz_seq) - 1):
            travel_t += abs(tz_seq[k + 1] - tz_seq[k])

        travel_by_team[t] = travel_t
        total_travel += travel_t

    return total_travel, travel_by_team


if model4.status == GRB.OPTIMAL:
    print("\n--- PART 4: OPTIMAL SCHEDULE FOUND ---")

    # Build optimized schedule dataframe
    solution_matches4 = []
    for (i, j, d), var in x4.items():
        if var.X > 0.5:
            solution_matches4.append({
                "Date": d,
                "Home": i,
                "Visitor": j,
                "TZ_Home": tz_map[i],
                "TZ_Visitor": tz_map[j]
            })

    df_opt = pd.DataFrame(solution_matches4)
    df_opt = df_opt.sort_values("Date")

    # Save optimized schedule
    df_opt.to_csv("optimized_schedule_part4.csv", index=False)
    print("Optimized schedule saved to optimized_schedule_part4.csv")

    # Compare travel: original vs optimized
    orig_total, orig_by_team = compute_tz_travel(df, teams, tz_map)
    new_total, new_by_team = compute_tz_travel(
        df_opt[["Date", "Home", "Visitor"]], teams, tz_map
    )

    print("\n=== PART 4: TIME-ZONE TRAVEL COMPARISON ===")
    print(f"Total TZ travel (original draft):  {orig_total}")
    print(f"Total TZ travel (optimized sched): {new_total}")
    print(f"Improvement: {orig_total - new_total} fewer TZ-steps\n")

    # Optional: print per-team comparison (nice for your PDF)
    for t in sorted(teams):
        print(f"{t:25s}  original: {orig_by_team[t]:3d}   optimized: {new_by_team[t]:3d}")

else:
    print("\nPart 4 model did not find an optimal solution. Status:", model4.status)



================= QUESTION 4 =================

Solving Part 4 model (minimize total time-zone travel)...
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[rosetta2] - Darwin 25.1.0 25B78)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1216 rows, 1264 columns and 6560 nonzeros
Model fingerprint: 0x3b540bd9
Variable types: 240 continuous, 1024 integer (1024 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 816 rows and 774 columns
Presolve time: 0.02s
Presolved: 400 rows, 490 columns, 1889 nonzeros
Variable types: 0 continuous, 490 integer (445 binary)
Found heuristic solution: objective 216.0000000
Found heuristic solution: objective 206.0000000
Found heuristic solution: objective 200.0000000

Root relaxation: objective 1.692983e+02, 764 iterations, 0.01 seconds (0.02 work u